In [1]:
import selenium
import time
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from bs4 import BeautifulSoup
import re
print('import finished')

import finished


In [2]:
#Accesss to trang web
driver = webdriver.Chrome()
url = "https://www.minhngoc.net.vn/ket-qua-xo-so/dien-toan-vietlott/mega-6x45/20-07-2016.html"
driver.get(url)

def GetURL():
    # Retrieve all HTML from Webpage to a BeautifulSoup Object then converts to HTML (this seems redundant!)
    page_source = BeautifulSoup(driver.page_source)
    html = page_source.prettify()

    # Find Date of Lottery Results Announcement
    soup = BeautifulSoup(html, 'html.parser')
    date_input = soup.find('input', {'id': 'getngaykqxs_1'})
    date_value = date_input['value']

    # Find Lottery-Results
    pattern = re.compile(r'finnish')
    results = page_source.find_all('div', attrs={'class': pattern})

    # Convert results from BeautifulSoup Object to HTML
    html_results = ''
    for result in results:
        html_results += str(result)

    # Convert html
    soup1 = BeautifulSoup(html_results, 'html.parser')
    values = [int(div.text) for div in soup1.find_all('div')][:6]

    # Create a dataframe
    df = pd.DataFrame(columns=['announce_date', 'lottery_win_number'])
    df.loc[len(df)] = [date_value, values]

    return df


In [9]:
# Iterate over pages
def scrape_data(driver):
    df_all_page = pd.DataFrame(columns=['announce_date', 'lottery_win_number'])
    for page in range(50):
        df_one_page = GetURL()
        sleep(2)
        next_button = driver.find_element(By.XPATH, '//*[@id="pagenav"]/table/tbody/tr/td/ul/li[6]/a')
        next_button.click()
        #There is an issue with the popup after next_button.click(). Current solution is manually close the popup. Needs further research.
        #pop_up = driver.find_element(By.XPATH, '//*[@id="dismiss-button"]/div/span')
        #pop_up.click()
        #pop-up xpath: //*[@id="dismiss-button"]/div/span
        df_all_page = pd.concat([df_all_page, df_one_page],ignore_index=True)
        sleep(2)
    return df_all_page

# Call the function to run the code
scraped_data = scrape_data(driver)
print(scraped_data)


   announce_date        lottery_win_number
0     20-07-2016   [2, 17, 33, 37, 38, 45]
1     21-07-2016   [2, 17, 33, 37, 38, 45]
2     22-07-2016    [3, 4, 14, 20, 25, 35]
3     23-07-2016    [3, 4, 14, 20, 25, 35]
4     24-07-2016   [1, 10, 16, 18, 23, 38]
5     25-07-2016   [1, 10, 16, 18, 23, 38]
6     26-07-2016   [1, 10, 16, 18, 23, 38]
7     27-07-2016  [14, 17, 21, 25, 31, 37]
8     28-07-2016  [14, 17, 21, 25, 31, 37]
9     29-07-2016    [3, 8, 13, 20, 30, 36]
10    30-07-2016    [3, 8, 13, 20, 30, 36]
11    31-07-2016    [2, 6, 19, 21, 35, 39]
12    01-08-2016    [2, 6, 19, 21, 35, 39]
13    02-08-2016    [2, 6, 19, 21, 35, 39]
14    03-08-2016   [7, 10, 25, 27, 29, 40]
15    04-08-2016   [7, 10, 25, 27, 29, 40]
16    05-08-2016    [3, 9, 13, 18, 24, 31]
17    06-08-2016    [3, 9, 13, 18, 24, 31]
18    07-08-2016    [3, 5, 17, 22, 30, 31]
19    08-08-2016    [3, 5, 17, 22, 30, 31]
20    09-08-2016    [3, 5, 17, 22, 30, 31]
21    10-08-2016   [6, 15, 16, 19, 27, 30]
22    11-08

In [10]:
# Save data to CSV
scraped_data.to_csv('sample.csv')

#Outcome has issue. Date format messed up. 2nd column return a list with the "[]" symbols.